#  PROPHET

In [3]:
%%time                   
import pandas as pd                             # прогноз без сезенности(Более подходящий)
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для товаров с меньше чем 2 непропущенными значениями 'y'

    # Создание и обучение модели Prophet для каждого товара
    model = Prophet()
    model.fit(sku_data[['ds', 'y']])

    # Создание DataFrame для прогноза на 60 дней вперед
    future = model.make_future_dataframe(periods=1200)
    forecast = model.predict(future)
    forecast['SKU'] = sku  # Добавление информации о товаре

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'SKU', 'yhat', 'yhat_upper']
all_forecasts[selected_columns].to_excel('прогноз_продаж_по_SKU.xlsx', index=False)


14:24:51 - cmdstanpy - INFO - Chain [1] start processing
14:24:51 - cmdstanpy - INFO - Chain [1] done processing
14:24:51 - cmdstanpy - INFO - Chain [1] start processing
14:24:52 - cmdstanpy - INFO - Chain [1] done processing
14:24:52 - cmdstanpy - INFO - Chain [1] start processing
14:24:52 - cmdstanpy - INFO - Chain [1] done processing


CPU times: total: 5.3 s
Wall time: 5.61 s


In [2]:
%%time               # прогноз с сезенностью( не нравится)
import pandas as pd
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для товаров с меньше чем 2 непропущенными значениями 'y'
    # Создание и обучение модели Prophet для каждого товара с учетом параметров
    model = Prophet(changepoint_prior_scale=0.5, seasonality_prior_scale=10)
    model.fit(sku_data[['ds', 'y']])

    # Создание DataFrame для прогноза на 60 дней вперед
    future = model.make_future_dataframe(periods=60)
    forecast = model.predict(future)
    forecast['SKU'] = sku  # Добавление информации о товаре

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'SKU', 'yhat', 'yhat_upper', 'yhat_lower']
all_forecasts[selected_columns].to_excel('прогноз_продаж.xlsx', index=False)


UsageError: Can't use statement directly after '%%time'!
